In [ ]:
###############################################################################################################
    #####################################################################################################
             ############  Text proprocessing, article by article (bag of words)  ############
    #####################################################################################################
###############################################################################################################

############################################################
 ##### TO DO FIRST TO MAKE THIS WORK: 
############################################################

########## 1. Change all xml files into txt: 
 # *** Go to the command prompt and type in:
   # A. cd\
   # B. cd\Users\jthibodeaux\python\Lib\site-packages\allofplos\allofplos_text
   # C. ren *.xml *.txt


############################################################
##########  TO DO SECOND TO MAKE THIS WORK: 
  ## This code reaches memory failure after about 70k articles for title and abstract only. 
  ## SO: Break allofplos corpus into 1/4ths 
       # Break into even smaller fractions if you're doing the entire text instead of title/abstract).
############################################################


In [ ]:
#####################################
 ##### The below opens / reads all of the text files from a folder into Python.
#####################################
import os
import pandas as pd
allLines = []
path = 'python/Lib/site-packages/allofplos/allofplos_text_5/'
  # NOTE: This means the 'allofplos_text' folder has to be a child folder of the folder this jupyter notebook is in.
fileList = os.listdir(path)
for file in fileList:
   file = open(os.path.join('python/Lib/site-packages/allofplos/allofplos_text_5/'+ file), 'r', encoding='utf-8')
   allLines.append(file.read())


####### This changes the file list into dois by taking off the .txt:
fileList = [x[:-4] for x in fileList]




#####################################
 ##### Specify how many chunks you want this iteration of your corpus broken further into:
    # This is to prevent memory failure AND to provide datasets with few columns (and thus a lower storage size).
    # Smaller chunks = larger bag_of_words file sizes BUT slower processing of TF-IDFs and related articles. 
#####################################
chunk_num = 18



#### Make sure all files were imported by checking length:
len(allLines)



In [ ]:
#################################################
 ##### BOTH OF THE BELOW FILTERS BY RESEARCH-ARTICLES *AND* TAKES OUT HTML LANGUAGE
#################################################

##############################
 ## Time difference:
    ##  #1 (ALL WORDS) takes about 3 articles per second. #2 (title & abstract only) takes about 30 articles per second. 
##############################


#################################################
##### 1. The below use ALL WORDS from the article:
#################################################

############################################################################ Disabled for now
#import re
#from bs4 import BeautifulSoup
#full_corpus = []
#research_article_list = []
#for i in range(0, len(allLines)):
#    if len(re.findall('article-type="research-article', allLines[i])) > 0:    
#        soup = BeautifulSoup(allLines[i], "xml")
#        text = soup.get_text(strip=False)
#        full_corpus.append(text)
#        research_article_list.append(fileList[i])





#################################################
##### 2. The below removes all text except title and abstract:
#################################################

##################
######## This allows you to get the title and abstract: 
#### NOTE: Using <abstract> xml tag works for most but not all; the below 3 methods gets them all:
#### You use split() to split the data by a particular string, then only use the string beforehand.
##Title
   # Start: <title-group>
   # End: <alt-title alt-title-type
##Abstract_1:
   # Start: </permissions>
   # End: </abstract>
      # Also, add on a second group of text if there's over 1 </abstract> after the </permissions>
##Abstract_2:
   # Start: </permissions>
   # End: <abstract abstract-type="summary">
##################
import re
from bs4 import BeautifulSoup
full_corpus = []
research_article_list = []
for i in range(0, len(allLines)):
    if len(re.findall('article-type="research-article', allLines[i])) > 0:    
        text = str(allLines[i])
        ## Titles look to be called similarly across allofplos (abstracts are more difficult, see below)
        title = text.split('<alt-title alt-title-type')[0]
        title = title.split('<title-group>')[1]
        ## This is an additional way to include abstracts BUT they aren't in every article:
        Another_abstract = 0
        if len(re.findall('</abstract></article-meta>', text)) > 0:
            abstract_2 = text.split('</abstract></article-meta>')[0]
            abstract_2 = abstract_2.split('</permissions>')[1]
            Another_abstract = Another_abstract + 1
        else:         # This is the main way to get the abstract:
            abstract = text.split('</permissions>')[1]
            abstract_1 = abstract.split('</abstract>')[0]
        ### Get rid of xml tags for the title :
             ## NOTE: In many cases, using BeautifulSoup on the abstract portion removes a lot of the abstract so I removed it (I remove related words in the below cell under 'to_remove' such as 'abstract'):
        soup = BeautifulSoup(title, "xml")
        title = soup.get_text(strip=False)
        ### If there's more than one </abstract> after permissions, add the part after that first </abstract> too.
           ## NOTE: included title twice to highlight it's importance.
        if Another_abstract > 0:
            text = title + " " + title + " " + abstract_2
        else:
            if len(re.findall('</abstract>', abstract)) > 1:
                abstract_3 = abstract.split('</abstract>')[1]
                text = title + " " + title + " " + abstract_1 + " " + abstract_3
            else:
                text = title + " " + title + " " + abstract_1
        full_corpus.append(text)
        research_article_list.append(fileList[i])




In [ ]:
###### Combined with below, this allows you to time how fast your code runs:
import time
start_time = time.time()


#####################################
 ##### Step 1: Create a loop to clean the articles (e.g. punctuation, upper cases, etc):
#####################################
import nltk
import re
import string
## The following 2 could be skipped if you want to keep the stopwords:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

new_corpus = []
for i in range(len(full_corpus)):
    temp = []
    temp = full_corpus[i].lower()
    temp = re.sub(r'\d+', '', temp)
    temp = temp.translate(str.maketrans("", "", string.punctuation))
    temp = temp.strip()
    ### The following three could be skipped if you want to keep the stop words:
    temp = word_tokenize(temp)
    stop_words = set(stopwords.words("english"))
    temp = [i for i in temp if not i in stop_words]
    ### Get rid of any words you think should be removed (eg uninformative common words and tags)
    to_remove = ["plos", "use", "using", "abstract", "sec", "title", "italic", "summary", "type", "toc", "bold", "relatedarticle", "page", "relatedarticletype", "article-title", "sup", "abstracttype", "reftypebibr", "xref", "contenttypepdf", "ref", "idsec", "summarytitl", "pthe", "yearyear", "abstracttypesummari", "volumevolum", "xlinktypesimplenam", "mixedcit", "rowspan", "xlinktypesimpl", "italicin", "extlinktypeuri", "titleresultstitl", "titlemethodstitl", "fpagefpag", "abstracttypetoc", "italicsital", "lpagelpag", "cdsupsup", "id", "titlemethod", "metaanalysi", "sourceplo", "namedcont", "articletitleth", "mimetypeimag", "commentdoiextlink", "italicmital", "fpageefpag", "abstractsec", "titleresultstitl", "findingstitl", "titlemethodstitl", "italicpitaliclt", "titlemethod", "pthese", "iii", "persongroup", "tgfβ", "ip", "extlink']", "extlink", "fpagefpag", "mimetypeimag", "lpagelpag", "cellsp']", "cellsp", "secsec", "titleresultstitl", "titlemethodstitl", "italicpital", "sectypehead", "link", "sec", "titleobjectivetitl", "tlr", "findingstitl", "ital", "supplement", "namedcont", "italica", "titlemethod", "methodstitl", "italicsital", "titlepurposetitl", "italicd", "titlemateri", "italicnital", "italict", "italicnamedcont", "italich", "titletri", "italiceital", "contenttypegenu", "titleresultstitl", "titlemethodstitl", "italicpital", "xlinktypesimpl", "titleobjectivetitl", "italicsital", "italiccital", "italiceital", "subtyp", "italicaital", "italicbital", "italicnital", "titlepurposetitl", "titlemateri", "italiccoliital", "italiclital", "italicpital", "unique", "name", "use", "would", "either", "introduction"]
    ## bad_col: Get rid of all words with 'X' in them (since beautiful soup doesn't work well on abstracts):
    bad_col_1 = re.compile("link:*")                       # NOTE: This matching on link bit is new.
    bad_col_2 = re.compile("abstract*")
    bad_col_3 = re.compile("title*")
    bad_col_4 = re.compile("xlink*")
    bad_col_5 = re.compile("label*")
    matches_1 = [i for i in temp if re.match(bad_col_1, i)]
    matches_2 = [i for i in temp if re.match(bad_col_2, i)]
    matches_3 = [i for i in temp if re.match(bad_col_3, i)]
    matches_4 = [i for i in temp if re.match(bad_col_4, i)]
    matches_5 = [i for i in temp if re.match(bad_col_5, i)]
    to_remove.extend(matches_1)
    to_remove.extend(matches_2)
    to_remove.extend(matches_3)
    to_remove.extend(matches_4)
    to_remove.extend(matches_5)
    temp = [i for i in temp if not i in to_remove]
    ### get rid of 1 and 2 letter words AND words with 20 characters or more:
    temp = [i for i in temp if (len(i) > 2) and (len(i) < 20) ]
    ### Stem the words (e.g. using and used becomes use)
    stemmer = PorterStemmer()
    temp = [stemmer.stem(i) for i in temp]
    new_corpus.append(str(temp))
    bad_col_4 = re.compile("xlink*")
    bad_col_5 = re.compile("label*")




#####################################
 ##### Step 2: Brake the article list into chunks (list of lists) of articles:
#####################################
import math

####### Calculate 'chunk_num' equal chunks and the remainder:
length = len(new_corpus)
size = math.floor(length / chunk_num)
remainder = length%chunk_num

####### Get an equal size chunk list of your corpus list (i.e. a list of a list):
corpus_chunks = {}
for i in range(chunk_num):
    temp_list = []
    for j in range(((size+(size*i)) - size), size+(size*i)):
        if (size+(size*i)) <= length:
            temp_list.append(new_corpus[j])
            corpus_chunks[i] = temp_list

####### Get the remainder of the list:
temp_list = []
for j in range(size*chunk_num, length):
    temp_list.append(new_corpus[j])
    corpus_chunks[chunk_num] = temp_list




#####################################
 ##### Step 3: Method to create a 'bag of words' (all unique words in the corpus and # of words in each article)
#####################################
from collections import Counter
import pandas as pd

def make_matrix(corpus):
    matrix = []
    # Create a list of all the words in this chunk of the corpus:
    unique_words = []
    unique_words = list(set(" ".join(corpus).split()))
    for i in range(len(corpus)):
        # Get all unique words across all articles in this chunk:
        bbb = []
        bbb = corpus[i].split()
        counter = Counter(bbb)
        # Turn the dictionary into a matrix row using the vocab.
        row = [counter.get(w, 0) for w in unique_words]
        matrix.append(row)
    df = pd.DataFrame(matrix)
    # Make the column names the words
    df.columns = unique_words
    # Exclude columns with less than n words metioned in this chunk
    df.drop([col for col, val in df.sum().iteritems() if val < 3], axis=1, inplace=True)
    return df



#####################################
 ##### Step 4: Use the main method from step 3 on each chunk of the corpus to get a list of 'bag of words' dataframes:
#####################################
dataframe_collection = {}
for i in range(len(corpus_chunks)):
    dataframe_collection[i] = make_matrix(corpus_chunks[i])





################################################################# 
 ##### Step 5: Create a dataframe with all words across lists. This allows you to:
    # A: Create a column sum dataframe (i.e. the sum of each word across the entire corpus).
    # B: Remove columns with under n words in this entire chunk to reduce the size of the dataframe:
################################################################# 
####################################
 ## Step 5a: This makes ALL dataframes in the list have the same columns (and in the same order):
       # NOTE FOR MODIFICATIONS: If needed, this will allow you to move the dataframe to SQL or some other program, union and create the TFIDF from there.
####################################
####### This loop gets all the columns (i.e. words) from all dataframes into one list:
full_col_list = []
for i in range(len(dataframe_collection)):
    temp = []
    temp = list(dataframe_collection[i].columns)
    full_col_list = full_col_list + temp

####### Make this list unique:
full_col_list = list(set(full_col_list))


####### Make an empty dataframe with all the columns (i.e. words) from all dataframes 'full_col_list'
import pandas as pd
main_df = pd.DataFrame(columns=full_col_list)


####### This concats the main_df dataframe (with all the columns) with each dataframe. It also turns all NaN into 0
  # Most important, this means all dataframes will have the same columns in the same order:
modified_collection = {}
for i in range(len(dataframe_collection)):
    modified_collection[i] = pd.concat([main_df, dataframe_collection[i]], ignore_index=True, sort=False)
    modified_collection[i] = modified_collection[i].fillna(0)




####################################
 ## Step 5b: Use column sums of each dataframe to remove columns with under n words in total:
####################################

##############################
 ##### 5b1. Create a dataframe where each row is the column sum of each dataframe.
##############################
df_colsum = pd.DataFrame(columns=full_col_list)
for i in range(len(modified_collection)):
    temp = []
    temp = modified_collection[i].sum(axis=0)
    df_colsum = df_colsum.append(temp,ignore_index=True)



##############################
 ##### 5b2. Sum the col sums across dataframes and remove all columns with under n words from ALL dataframes.
   ## I use 5 words here (for 1/4th of allofplos & only title/abstract, probably use a much higher number for full text).
      ## In Step 7 I will remove words with a higher number after all iterations are combined.
##############################
n_words = 5

all_col_sum = df_colsum.sum(axis=0)
all_col_sum = all_col_sum[all_col_sum < n_words]
all_col_sum = list(all_col_sum.index)


####### This is your full word sum (with words under n dropped)
df_colsum = df_colsum.drop(all_col_sum, axis=1)


####### This is your full bag of words with ALL words (as a list of dataframes (with words under n dropped):
for i in range(len(modified_collection)):
    modified_collection[i] = modified_collection[i].drop(all_col_sum, axis=1)


####### This is your full bag of words with only words for each chunk and words under 5 dropped (saves disk space):
for i in range(len(dataframe_collection)):
    df_cols = list(dataframe_collection[i].columns)
    for j in all_col_sum:
        if j in df_cols:
            dataframe_collection[i] = dataframe_collection[i].drop(j, axis=1)






################################################################# 
 ##### Step 6. Add in rownames of the DOIs (either in dataframe_collection or modified_collection depending on which you want to use)
################################################################# 

################### DISABLED FOR NOW:
####### For modified_collection: Use modified_collection if you want every dataframe to have ALL words in the corpus as columns (i.e. same columns for all dataframe)
#count = 0
#for i in range(len(modified_collection)):
#    modified_collection[i].index = research_article_list[count : (count+len(modified_collection[i]))]
#    count = count + len(modified_collection[i])

####### For dataframe_collection (bag_of_words with only words for each chunk)
count = 0
for i in range(len(dataframe_collection)):
    dataframe_collection[i].index = research_article_list[count : (count+len(dataframe_collection[i]))]
    count = count + len(dataframe_collection[i])



###### Combined with above, this allows you to time how fast your code runs:
print("My program took", time.time() - start_time, "to run")


In [ ]:
#########################################################################
 ####################################################################
  ########## Step 7: Stop here and download datasets
     ### 'modified_collection' here is the dataframe list you should save for future use (since TF-IDF will change over time but this bag of words list of dataframes will remain constant).
 ####################################################################
#########################################################################

################################################################# 
 ##### Step 7: Export list of data frames 'modified_collection' for future use.
################################################################# 

##########################
####### Step 7a: Export the chunks of bags of words:
##########################

###################    *** This is disable for now ***
####### 7a1: Modified_collection contains All words in (and same columns for) each dataframe: Save to file
  ## NOTE: Because it's a list of dataframes, you'll need to save as n different csvs:
#for i in range(len(modified_collection)):
#    modified_collection[i].to_csv(rf"TFIDFs_dataset_{i}.csv")

####### 7a2: ALMOST ALWAYS USE THIS: Only columns (words) that exist in each respective chunk: Save to file
for i in range((len(dataframe_collection))):
    count = i + 80
    dataframe_collection[i].to_csv(rf"allofplos_bagofwords/smaller_TFIDFs_dataset_{count}.csv")



##########################
####### Step 7b: Export the sum of each word in the corpus:
##########################
full_word_count = pd.DataFrame(df_colsum.sum(axis=0)).T
full_word_count.to_csv(rf"Summed_allofplos_BagOfWords_5.csv")
      ### NOTE: If you do a huge file, write the CSV as a zip file by using: full_word_count.to_csv(rf"full_BagOfWordszipped.csv", compression='gzip')




In [ ]:

####################################################################
 ###### Do the above for as many sets of files you broke the entire corpus into
   # NOTE: I broke it into 5 sets of around 55k files each for title x2 and abstract
 ###### When done with those do the below (step 8)
####################################################################


In [ ]:
#########################################################################
 ####################################################################
  ########## Step 8: Combining different iterations and removing more unwanted words.
     ### AFTER CREATING ALL CSVS: Import and modify all bag of words and column sums
 ####################################################################
######################################################################### 

import os
import pandas as pd

####### Imports sum of all words for the 5 iterations:
import_sum_1 = pd.read_csv("Summed_allofplos_BagOfWords.csv")
import_sum_2 = pd.read_csv("Summed_allofplos_BagOfWords_2.csv")
import_sum_3 = pd.read_csv("Summed_allofplos_BagOfWords_3.csv")
import_sum_4 = pd.read_csv("Summed_allofplos_BagOfWords_4.csv")
import_sum_5 = pd.read_csv("Summed_allofplos_BagOfWords_5.csv")

####### Sum the sum of all words across the 5 iterations
full_word_count = pd.DataFrame()
full_word_count = pd.concat([import_sum_1, import_sum_2], ignore_index=True, sort=False)
full_word_count = pd.concat([full_word_count, import_sum_3], ignore_index=True, sort=False)
full_word_count = pd.concat([full_word_count, import_sum_4], ignore_index=True, sort=False)
full_word_count = pd.concat([full_word_count, import_sum_5], ignore_index=True, sort=False)
full_word_count = full_word_count.fillna(0)
full_word_count = pd.DataFrame(full_word_count.sum(axis=0))


####### Drop columns with a count of n & under (I eliminated words used under 25 times total in the entire corpus):
cols_to_drop = full_word_count[full_word_count.values <= 25]
cols_to_drop = cols_to_drop.index
cols_to_drop = list(set(cols_to_drop)) # De-duplicate.
full_word_count = full_word_count.T
full_word_count = full_word_count.drop(cols_to_drop, axis=1)




######################################################
########## OPTIONAL/MESSY: Use this to find common words you want removed from all of the bag_of_words
  #### IF DESIRED, TAKE THIS CODE CHUNK AND USE IN A DIFFERENT CELL:
######################################################
################################################################### DISABLED FOR NOW:
#col_names = full_word_count.columns
#g = full_word_count.sort_values(by=[col_names[0]], ascending=False)
#g[0:50] # I went until 2k, 50 at a time. 

####### Use the below to remove new unwanted common words (I got these by looking through the common words (see just above) and selecting the ones I wanted removed):
#to_remove = ["'plos,'", "'use',", "'using',", "'abstract',", "'sec',", "'title',", "'italic',", "'summary',", "'type',", "'toc',", "'bold',", "'relatedarticle',", "'page',", "'relatedarticletype',", "'article-title',", "'sup',", "'abstracttype',", "'reftypebibr',", "'xref',", "'contenttypepdf',", "'ref',", "'idsec',", "'summarytitl',", "'pthe',", "'yearyear',", "'abstracttypesummari',", "'volumevolum',", "'xlinktypesimplenam',", "'mixedcit',", "'rowspan',", "'xlinktypesimpl',", "'italicin',", "'extlinktypeuri',", "'titleresultstitl',", "'titlemethodstitl',", "'fpagefpag',", "'abstracttypetoc',", "'italicsital',", "'lpagelpag',", "'cdsupsup',", "'id',", "'titlemethod',", "'metaanalysi',", "'sourceplo',", "'namedcont',", "'articletitleth',", "'mimetypeimag',", "'commentdoiextlink',", "'italicmital',", "'fpageefpag',", "'abstractsec',", "'titleresultstitl',", "'findingstitl',", "'titlemethodstitl',", "'italicpitaliclt',", "'titlemethod',", "'pthese',", "'iii',", "'persongroup',", "'tgfβ',", "'ip',", "'extlink']',", "'extlink',", "'fpagefpag',", "'mimetypeimag',", "'lpagelpag',", "'cellsp']',", "'cellsp',", "'secsec',", "'titleresultstitl',", "'titlemethodstitl',", "'italicpital',", "'sectypehead',", "'link',", "'sec',", "'titleobjectivetitl',", "'tlr',", "'findingstitl',", "'ital',", "'supplement',", "'namedcont',", "'italica',", "'titlemethod',", "'methodstitl',", "'italicsital',", "'titlepurposetitl',", "'italicd',", "'titlemateri',", "'italicnital',", "'italict',", "'italicnamedcont',", "'italich',", "'titletri',", "'italiceital',", "'contenttypegenu',", "'titleresultstitl',", "'titlemethodstitl',", "'italicpital',", "'xlinktypesimpl',", "'titleobjectivetitl',", "'italicsital',", "'italiccital',", "'italiceital',", "'subtyp',", "'italicaital',", "'italicbital',", "'italicnital',", "'titlepurposetitl',", "'titlemateri',", "'italiccoliital',", "'italiclital',", "'titleresultstitl',", "'titlemethodstitl',", "'italicpital',", "'xlinktypesimpl',"]
#to_remove = list(set(to_remove)) # De-duplicate.
   # NOTE: The columns end up having a ' then a ,' around them. SO you either add in ' and ,' for the words in to_remove OR remove those punctuations from the column names.

#aa = list(full_word_count.columns)
#for i in to_remove:
#    if i in aa:
#        full_word_count = full_word_count.drop(i, axis=1)




############## Save a file full word count across the entire corpus to file:
full_word_count.to_csv(r"Full_allofplos_bagofwords.csv")
   # After this, you can delete all 4 'Summed_allofplos_BagOfWords' iterations and just use this as your full summed bag_of_words.





############## Import and drop columns with a count under n words (and commons words you want removed), then (re)save these files:
path = "allofplos_bagofwords/"
fileList = os.listdir(path)
temp = {}
for i in range(len(fileList)):
    temp[i] = pd.read_csv("allofplos_bagofwords/"+fileList[i])
    dois = temp[i].iloc[:,0]
    bb = list(temp[i].columns)
    for j in cols_to_drop:
        if j in bb:
            temp[i] = temp[i].drop(j, axis=1)
#    for j in to_remove:                    ############# DISABLED FOR NOW: ENABLE IF YOU ADD THE 'TO_REMOVE' LIST
#        if j in bb:
#            temp[i] = temp[i].drop(j, axis=1)
    temp[i].index = dois
    temp[i].to_csv(rf"final_allofplos_bagofwords/smaller_TFIDFs_dataset_{i}.csv")



In [ ]:

###############################################################
 ###### See file 'Create_related_article_lists_for_allofplos' for doing TF-IDFs & related articles.
###############################################################

